In [ ]:
import os,sys
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from keras.models import Sequential, Model, load_model
from keras.layers import *
from keras.callbacks import *
from keras.utils import np_utils
from keras.optimizers import SGD, Adam
from keras.preprocessing.image import ImageDataGenerator

In [ ]:
epochs=32
batch_size=512
images=[]
labels= []
name= []

In [3]:
def read_images_labels(path,i):
    for file in os.listdir(path):
        abs_path = os.path.abspath(os.path.join(path, file))
        if os.path.isdir(abs_path):
            i+=1
            temp = os.path.split(abs_path)[-1]
            name.append(temp)
            read_images_labels(abs_path,i)
            amount = int(len(os.listdir(path)))
            sys.stdout.write('\r'+'>'*(i)+' '*(amount-i)+'[%s%%]'%(i*100/amount)+temp)
        else:
            if file.endswith('.jpg'):
                image=cv2.resize(cv2.imread(abs_path),(64,64))
                images.append(image)
                labels.append(i-1)
    return images, labels , name

In [ ]:
def read_main(path):
    images, labels ,name = read_images_labels(path,i=0)
    images = np.array(images,dtype=np.float32)/255
    labels = np_utils.to_categorical(labels, num_classes=20)
    np.savetxt('name.txt', name, delimiter = ' ',fmt="%s")
    return images, labels
#labels=3，經過np_utils.to_categorical，會轉換為 labels= [0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]

images, labels=read_main('/content/drive/MyDrive/train')
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.1  )


>[100.0%]characters-20

In [ ]:
print('\r')
print("-----------------------")
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)
print("-----------------------")



-----------------------
(9491, 64, 64, 3)
(9491, 20)
(1055, 64, 64, 3)
(1055, 20)
-----------------------


In [ ]:
model = Sequential()
model.add(Conv2D(64, kernel_size=3 , padding='same',activation='relu', input_shape=X_train.shape[1:]))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.5))

model.add(Conv2D(128, kernel_size=3, padding='same',activation='relu'))
model.add(Conv2D(128, kernel_size=3, padding='same',activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.5))

model.add(Conv2D(256, kernel_size=3, padding='same',activation='relu'))
model.add(Conv2D(256, kernel_size=3, padding='same',activation='relu'))
model.add(Conv2D(256, kernel_size=3, padding='same',activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.5))
model.add(Flatten())
model.add(Dense(512,activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(20,activation='softmax'))
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 64, 64, 64)        1792      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 32, 32, 64)        0         
_________________________________________________________________
dropout (Dropout)            (None, 32, 32, 64)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 32, 32, 128)       73856     
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 32, 32, 128)       147584    
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 16, 16, 128)       0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 16, 16, 128)       0

In [ ]:
sgd = SGD(lr=0.01, momentum=0.9, decay=1e-6, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer='adam',metrics=['accuracy'])

datagen = ImageDataGenerator(zoom_range=0.1,width_shift_range=0.05,height_shift_range=0.05,horizontal_flip=True)
datagen.fit(X_train)

In [ ]:
file_name=str(epochs)+'_'+str(batch_size)
model.fit_generator(datagen.flow  (X_train, y_train, batch_size=batch_size),
                    steps_per_epoch=epochs, epochs=epochs,
                    validation_data = (X_test, y_test ), verbose = 1)

Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/32
19/32 [================>.............] - 702s 37s/step - loss: 2.6949 - accuracy: 0.1565 - val_loss: 2.4038 - val_accuracy: 0.1185


In [ ]:
model.save('h5'+file_name+'.h5')
score = model.evaluate(X_test, y_test, verbose=1)
print(score)

33/33 [==============================] - 23s 689ms/step - loss: 2.4038 - accuracy: 0.1185
[2.403838634490967, 0.11848340928554535]


In [ ]:
import cv2
import numpy as np
from keras.models import load_model
import pandas as pd

def read_images(path):
    images=[]
    for i in range(990):
        image=cv2.resize(cv2.imread(path+str(i+1)+'.jpg'),(64,64))
        images.append(image)

    images=np.array(images,dtype=np.float32)/255
    return images

def transform(listdir,label,lenSIZE):
    label_str=[]
    for i in range (lenSIZE):
        temp=listdir[label[i]]
        label_str.append(temp)

    return label_str


In [ ]:
images = read_images('/content/drive/MyDrive/test/test/')
model = load_model('h532_512.h5')

predict = model.predict_classes(images, verbose=1)
print(predict)
label_str=transform(np.loadtxt('name.txt',dtype='str'),predict,images.shape[0])

df = pd.DataFrame({"character": label_str})
df.index = np.arange(1, len(df) + 1)
df.index.names = ['id']
df.to_csv('Predictions.csv')

Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
31/31 [==============================] - 19s 629ms/step
[7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7
 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7
 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7
 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7
 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7
 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7
 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7
 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7
 7 7 7 7 7